## **snunlp/kr-electra/generator를 이용한 데이터 증강**
made by eyeol

### 0. 배경 설명

문장의 일부 토큰을 유의어로 교체하여 데이터를 증강해보겠다는 아이디어가 팀내에서 나왔음 </br>

다른 팀원들은 w2v과 BERT를 이용하여 유의어 교체를 시도했하였고, </br>
나는 그것들과 겹치지 않는 방향으로 데이터 증강 방법을 고민해봄

snunlp/kr-electra/generator는 electra 모델을 한국어로 학습한 pre-trained 모델로, </br>
pre-train할 때 MaskedLM을 사용하기 때문에 유의어 생성에도 유리하고

STS task에서 주력으로 쓰던 snunlp/kr-electra/discriminator와 </br>
동일한 vocab을 사용한다는 점도 강점이 될 것이라고 봤다. </br>

둘다 mecab-ko라는 형태소 분석기로 tokenizing하여 만든 vocab을 사용하기에, </br>
사전 학습 당시, 문장의 맥락을 읽는 방법을 비슷하게 학습했을 것이라 생각

### 1. 준비

In [3]:
import random
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForMaskedLM

from torch.optim import AdamW
from transformers import get_scheduler



c:\Users\eyeol\miniconda3\envs\sts\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def set_seed(seed):
    random.seed(seed)  # Python random 시드 고정
    np.random.seed(seed)  # Numpy 시드 고정
    torch.manual_seed(seed)  # PyTorch 시드 고정 (CPU)
    torch.cuda.manual_seed_all(seed)  # PyTorch 시드 고정 (모든 GPU)
    
    # CUDA 비결정적 동작 방지 (재현성을 높이기 위해)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ipynb에서는 cell이 바뀌면 seed 정보가 날라가기 때문에
# 학습을 실행하는 셀에서 set.seed(42)를 실행해야 한다
    

### 2. 모델 불러오기 및 학습

그냥 불러온 generator로 유의어를 생성했을 때는 데이터 품질이 너무 좋지 않았다 </br>
특정 토큰이 #이나 , 처럼 원래의 문맥과 많이 달라지는 경우가 많았음 </br>

그래서 generator를 ElectraForMaskedLM으로 불러오고 </br>
train.csv의 sentence_1, sentence_2로 학습시켜서 train set에 오버피팅(?)시켜봤음

In [5]:
# 모델 및 토크나이저 불러오기
model = ElectraForMaskedLM.from_pretrained("snunlp/KR-ELECTRA-generator")
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-ELECTRA-generator")

In [6]:
# 모델 구조 확인
model

ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=768, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featur

In [7]:
class SentencesDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['sentence']
        encoding = self.tokenizer(sentence, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask}


#### train set을 MaskedLM이 학습할 수 있는 형태로 바꾸기

In [8]:
df = pd.read_csv("../raw/train.csv")

In [9]:
sentences = df["sentence_1"].tolist() + df["sentence_2"].tolist()
df_new = pd.DataFrame(sentences, columns=["sentence"])
df_new.to_csv("./v2/combined_sentences.csv", index=False)

#### dev set을 MaskedLM이 평가에 사용할 수 있는 형태로 바꾸기

In [10]:
df2 = pd.read_csv("../raw/dev.csv")

In [11]:
sentences2 = df2["sentence_1"].tolist() + df2["sentence_2"].tolist()
df_new2 = pd.DataFrame(sentences2, columns=["sentence"])
df_new2.to_csv("./v2/combined_sentences_dev.csv", index=False)

In [12]:
# MLM을 위한 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15  # 15% 마스킹
)

In [13]:
dataset = SentencesDataset(df_new, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=data_collator)

#### 학습 시작

In [14]:
# 손실 함수와 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습을 위한 스케줄러 설정 (선택 사항, learning rate 스케줄링)
num_epochs = 4
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [15]:
import torch
print(torch.__version__)  # 설치된 PyTorch 버전 확인
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인


2.3.1+cu121
True


In [16]:
# 모델을 GPU로 이동 (가능한 경우)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=768, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featur

In [17]:
set_seed(42)

# 모델 학습 루프
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    for batch in dataloader:
        # 배치 데이터를 GPU로 이동
        batch = {k: v.to(device) for k, v in batch.items()}

        # 모델에 입력하고 출력 계산
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["input_ids"])
        
        # 손실 계산 (MLM에서 labels는 input_ids와 동일, 마스크된 토큰에 대해서만 손실을 계산)
        loss = outputs.loss
        
        # 역전파를 통해 그래디언트 계산
        loss.backward()

        # 옵티마이저를 통해 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 업데이트 (선택 사항)
        lr_scheduler.step()

        # 옵티마이저의 그래디언트 초기화
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs} 완료. Loss: {loss.item()}")

Epoch 1/4 완료. Loss: 0.01738937944173813
Epoch 2/4 완료. Loss: 0.010741570964455605
Epoch 3/4 완료. Loss: 0.011487413197755814
Epoch 4/4 완료. Loss: 0.011057838797569275


#### 검증

In [18]:
# 1. dev.csv 파일을 불러와서 Dataset과 DataLoader 생성
df_dev = pd.read_csv("./v2/combined_sentences_dev.csv")

# Dev Dataset 생성
dev_dataset = SentencesDataset(df_dev, tokenizer)

# Dev DataLoader 생성
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

In [19]:
# 2. 모델 평가 함수 정의
def evaluate(model, dataloader):
    model.eval()  # 모델을 평가 모드로 설정 (드롭아웃 비활성화)
    
    total_loss = 0
    total_accuracy = 0
    total_samples = 0
    
    with torch.no_grad():  # 평가 시에는 그래디언트를 계산하지 않음
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # 모델 출력과 손실 계산
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["input_ids"])
            loss = outputs.loss
            logits = outputs.logits
            
            # 손실 값 누적
            total_loss += loss.item()
            
            # 예측된 토큰과 실제 토큰 비교하여 정확도 계산
            predictions = torch.argmax(logits, dim=-1)
            labels = batch["input_ids"]
            
            # 마스크된 부분만 정확도 계산 (loss에서는 mask로 손실을 계산했으므로, 동일한 부분만 정확도를 계산)
            mask = labels != tokenizer.pad_token_id  # 패딩된 부분을 제외
            correct_predictions = (predictions == labels) & mask
            accuracy = correct_predictions.sum().item() / mask.sum().item()
            total_accuracy += accuracy * mask.sum().item()
            total_samples += mask.sum().item()

    # 평균 손실과 정확도 계산
    avg_loss = total_loss / len(dataloader)
    avg_accuracy = total_accuracy / total_samples
    return avg_loss, avg_accuracy

In [20]:
# 3. 평가 실행
dev_loss, dev_accuracy = evaluate(model, dev_dataloader)

In [21]:
# 4. 평가 결과 출력
print(f"Dev Loss: {dev_loss:.4f}, Dev Accuracy: {dev_accuracy:.4f}")

Dev Loss: 0.0109, Dev Accuracy: 0.9325


### 3. 유의어 생성 및 토큰 대체

In [22]:
# 1. train.csv 파일을 불러옴
df = pd.read_csv("../raw/train.csv")

In [23]:
# 모델을 평가 모드로 설정
model.eval()

ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=768, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featur

In [24]:
# GPU 사용 설정 (선택 사항)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=768, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featur

In [25]:
# 3. 유의어로 대체할 토큰 선택 및 교체하는 함수
def get_synonym_replacement(sentence, prob=0.15):
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.encode(sentence, return_tensors="pt").to(device)

    # 마스크할 토큰들 선택 (확률적으로)
    masked_indices = [
        i for i in range(1, len(tokens) - 1)  # [CLS]와 [SEP]를 제외한 중간 토큰들만 대상
        if random.random() < prob
    ]
    
    if not masked_indices:  # 마스킹할 토큰이 없으면 원본 문장 반환
        return sentence
    
    # 마스킹된 토큰 생성
    for idx in masked_indices:
        tokens[idx] = "[MASK]"
    
    # 마스킹된 문장 생성
    masked_sentence = tokenizer.convert_tokens_to_string(tokens)
    masked_input = tokenizer(masked_sentence, return_tensors="pt").to(device)
    
    # 마스크된 위치에 대한 토큰 예측
    with torch.no_grad():
        outputs = model(**masked_input)
    
    # 마스크된 위치에서 예측된 토큰 중 상위 5개를 추출하고 하나를 랜덤하게 선택
    predictions = outputs.logits
    
    # 마스크된 위치에서 예측된 토큰 중 최상위 1개를 추출
    for idx in masked_indices:
        token_logits = predictions[0, idx]
        top_token = torch.argmax(token_logits).item()  # 최상위 토큰 1개 선택
        replacement_token = tokenizer.decode([top_token]).strip()
        tokens[idx] = replacement_token

    # 문장 재구성
    augmented_sentence = tokenizer.convert_tokens_to_string(tokens)
    return augmented_sentence

In [26]:
# 4. 기존의 데이터를 유지하면서 sentence_1과 sentence_2만 유의어로 교체한 데이터를 추가
def augment_data(df):
    augmented_data = []

    # 기존 데이터를 증강하는 과정
    for i in range(len(df)):
        original_sentence_1 = df.loc[i, "sentence_1"]
        original_sentence_2 = df.loc[i, "sentence_2"]

        # 각 문장을 유의어로 대체
        augmented_sentence_1 = get_synonym_replacement(original_sentence_1)
        augmented_sentence_2 = get_synonym_replacement(original_sentence_2)

        # 증강된 문장과 기존 id, source, label, binary-label 추가
        augmented_data.append({
            "id": df.loc[i, "id"],
            "sentence_1": augmented_sentence_1,
            "sentence_2": augmented_sentence_2,
            "source": df.loc[i, "source"],
            "label": df.loc[i, "label"],
            "binary-label": df.loc[i, "binary-label"]
        })
    
    # 기존 데이터프레임을 그대로 유지하고 증강된 데이터 추가
    df_augmented = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)
    
    return df_augmented

In [27]:
# 6. 증강된 데이터를 얻고 CSV로 저장
df_augmented = augment_data(df)
df_augmented.to_csv("./v2/full_augmented_train.csv", index=False)
print("증강된 데이터가 'augmented_train.csv'로 저장되었습니다.")

증강된 데이터가 'augmented_train.csv'로 저장되었습니다.


In [ ]:
## 부분 증강을 위한 함수

# 기존의 데이터를 유지하면서 label < 1인 데이터에 대해서만 sentence_1과 sentence_2를 유의어로 교체한 데이터를 추가
def augment_partial_data(df):
    augmented_data = []

    # 기존 데이터를 증강하는 과정 (label < 1인 데이터에 대해서만)
    for i in range(len(df)):
        original_sentence_1 = df.loc[i, "sentence_1"]
        original_sentence_2 = df.loc[i, "sentence_2"]
        label = df.loc[i, "label"]

        # label 값이 1 미만인 데이터에만 유의어 증강 적용
        if label < 1:
            # 각 문장을 유의어로 대체
            augmented_sentence_1 = get_synonym_replacement(original_sentence_1)
            augmented_sentence_2 = get_synonym_replacement(original_sentence_2)

            # 증강된 문장과 기존 id, source, label, binary-label 추가
            augmented_data.append({
                "id": df.loc[i, "id"],
                "sentence_1": augmented_sentence_1,
                "sentence_2": augmented_sentence_2,
                "source": df.loc[i, "source"],
                "label": df.loc[i, "label"],
                "binary-label": df.loc[i, "binary-label"]
            })
    
    # 기존 데이터프레임을 그대로 유지하고 증강된 데이터 추가
    df_augmented = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)
    
    return df_augmented